In [227]:
import openmeteo_requests

from datetime import timedelta
import requests_cache
import pandas as pd
from retry_requests import retry

import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

import ast
import json

def extract_data(row):
    if row['Indoor'] == 'No':
        try: 
            json_data =  row['weather_info']
            start_date = str(row['game_date'].strftime('%Y-%m-%d')) + str(row['game_date'].strftime('T%H:%M'))
            exact_date = str(row['rounded_exact_time'].strftime('%Y-%m-%d')) + str(row['rounded_exact_time'].strftime('T%H:%M'))

            # find the index where 'time' matches 'roundup_start_date'
            start_index = json_data["hourly"]["time"].index(start_date)
            exact_index = json_data["hourly"]["time"].index(exact_date)

            # extract data based on the index
            temperature = json_data["hourly"]["temperature_2m"][exact_index]
            precipitation = np.sum(json_data["hourly"]["precipitation"][start_index-2:exact_index])
            windspeed = json_data["hourly"]["windspeed_10m"][exact_index]
            windgusts = json_data["hourly"]["windgusts_10m"][exact_index]
            elevation = json_data["elevation"]

            return pd.Series({'elevation':elevation, 'temperature': temperature, 'precipitation': precipitation, 'windspeed': windspeed, 'windgusts': windgusts})
        except:
            return None
    else: 
        json_data = row['weather_info']
        elevation = json_data["elevation"]
        return pd.Series({'elevation': elevation, 'temperature': 70, 'precipitation': 0, 'windspeed': 0, 'windgusts': 2.5})


Import data

In [228]:
# sked = pd.read_csv("schedule.csv")
# sked['Date'] = pd.to_datetime(sked['Date'])
# sked['Time'] = pd.to_datetime(sked['Time'], format='%I:%M %p').dt.

## now with nflfastr, I can just use the times from the actual data. 
## so I need to merge first, then datetime the columns

year = 2022
pbp = pd.read_csv(f"kicks_{year}_{year + 1}.csv")
pbp['game_date'] = pd.to_datetime(pbp['game_date'])
pbp['start_time'] = pbp.apply(lambda row: pd.to_datetime(row['start_time'], format="%m/%d/%y, %H:%M:%S").tz_localize("Etc/GMT+5").time(), axis = 1)
pbp['time_of_day'] = pbp.apply(lambda row: pd.to_datetime(row['time_of_day']).tz_convert('Etc/GMT+4'), axis = 1)

pbp.head()

,play_id,game_id,Number,Kicker,Yards,Outcome,posteam,defteam,game_date,qtr,...,away_team,start_time,stadium,time_of_day,Team,Abbr,Latitude,Longitude,Indoor,Turf
0,790,2022_01_BAL_NYJ,9,J. Tucker,24,Good,BAL,NYJ,2022-09-11,1,...,BAL,13:05:56,MetLife Stadium,2022-09-11 13:37:20-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf
1,1216,2022_01_BAL_NYJ,6,G. Zuerlein,45,No Good,NYJ,BAL,2022-09-11,2,...,BAL,13:05:56,MetLife Stadium,2022-09-11 13:55:43.073000-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf
2,1919,2022_01_BAL_NYJ,6,G. Zuerlein,45,Good,NYJ,BAL,2022-09-11,2,...,BAL,13:05:56,MetLife Stadium,2022-09-11 14:29:02.307000-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf
3,1153,2022_01_BUF_LA,2,T. Bass,41,Good,BUF,LA,2022-09-08,2,...,BUF,20:23:17,SoFi Stadium,2022-09-08 21:05:14-04:00,Los Angeles Chargers,LAC,33.953587,-118.339630,Yes,Turf
4,1888,2022_01_BUF_LA,8,M. Gay,57,Good,LA,BUF,2022-09-08,2,...,BUF,20:23:17,SoFi Stadium,2022-09-08 21:37:23-04:00,Los Angeles Chargers,LAC,33.953587,-118.339630,Yes,Turf


Round up the time

In [229]:
def round_time(time):
    if time.minute > 0:
        new_hour = (time.hour + 1) % 24
        time = time.replace(hour = new_hour,minute=0, second=0)

        if new_hour == 0:
            time += timedelta(days=1)
    return time

pbp['rounded_start_time'] = pbp['start_time'].apply(round_time)
pbp['rounded_exact_time'] = pbp['time_of_day'].apply(round_time)

pbp.head()

,play_id,game_id,Number,Kicker,Yards,Outcome,posteam,defteam,game_date,qtr,...,stadium,time_of_day,Team,Abbr,Latitude,Longitude,Indoor,Turf,rounded_start_time,rounded_exact_time
0,790,2022_01_BAL_NYJ,9,J. Tucker,24,Good,BAL,NYJ,2022-09-11,1,...,MetLife Stadium,2022-09-11 13:37:20-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf,14:00:00,2022-09-11 14:00:00-04:00
1,1216,2022_01_BAL_NYJ,6,G. Zuerlein,45,No Good,NYJ,BAL,2022-09-11,2,...,MetLife Stadium,2022-09-11 13:55:43.073000-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf,14:00:00,2022-09-11 14:00:00.073000-04:00
2,1919,2022_01_BAL_NYJ,6,G. Zuerlein,45,Good,NYJ,BAL,2022-09-11,2,...,MetLife Stadium,2022-09-11 14:29:02.307000-04:00,New York Giants,NYG,40.812194,-74.076983,No,Turf,14:00:00,2022-09-11 15:00:00.307000-04:00
3,1153,2022_01_BUF_LA,2,T. Bass,41,Good,BUF,LA,2022-09-08,2,...,SoFi Stadium,2022-09-08 21:05:14-04:00,Los Angeles Chargers,LAC,33.953587,-118.339630,Yes,Turf,21:00:00,2022-09-08 22:00:00-04:00
4,1888,2022_01_BUF_LA,8,M. Gay,57,Good,LA,BUF,2022-09-08,2,...,SoFi Stadium,2022-09-08 21:37:23-04:00,Los Angeles Chargers,LAC,33.953587,-118.339630,Yes,Turf,21:00:00,2022-09-08 22:00:00-04:00


Weather data pull

In [230]:
unique_games = pbp[['game_id', 'Latitude', 'Longitude', 'game_date', 'start_time']].drop_duplicates().reset_index()
unique_games['weather_info'] = None

for i in tqdm(range(len(unique_games)), desc="Fetching Weather Data"):
    game = unique_games.iloc[i]
    lat = game['Latitude']
    lon = game['Longitude']
    start_date = pd.to_datetime(game["game_date"]).date()
    next_date = pd.to_datetime(game["game_date"]).date() + timedelta(days = 1)

    response = requests.get(f'https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={start_date}&end_date={next_date}&timezone=EST&temperature_unit=fahrenheit&hourly=temperature_2m,apparent_temperature,precipitation,rain,snowfall,weathercode,windspeed_10m,winddirection_10m,windgusts_10m')
    unique_games.at[i, 'weather_info'] = response.json()

pbp = pbp.merge(unique_games[['game_id', 'weather_info']], on='game_id', how='left')
pbp.columns

Fetching Weather Data:   0%|          | 0/282 [00:00<?, ?it/s]

Fetching Weather Data: 100%|██████████| 282/282 [03:03<00:00,  1.54it/s]


Index(['play_id', 'game_id', 'Number', 'Kicker', 'Yards', 'Outcome', 'posteam',
       'defteam', 'game_date', 'qtr', 'quarter_seconds_remaining',
       'yardline_100', 'desc', 'home_team', 'away_team', 'start_time',
       'stadium', 'time_of_day', 'Team', 'Abbr', 'Latitude', 'Longitude',
       'Indoor', 'Turf', 'rounded_start_time', 'rounded_exact_time',
       'weather_info'],
      dtype='object')

Dataframe the weather for each game

In [232]:
new_col = pbp.apply(extract_data, axis=1)
test = pd.concat([pbp, new_col], axis=1)

pbp = test[['play_id', 'game_id', 'Number', 'Kicker', 'Yards', 'Outcome', 'posteam',
       'defteam', 'game_date', 'qtr', 'quarter_seconds_remaining',
       'yardline_100', 'desc', 'home_team', 'away_team', 'start_time',
       'stadium', 'time_of_day', 'Team', 'Latitude', 'Longitude',
       'Indoor', 'Turf', 'elevation', 'temperature', 'precipitation',
       'windspeed', 'windgusts']]

Merge kicks with schedule by teams and date

In [233]:
kicks = pd.read_csv("STUD Field Goal Tracking - kicks_2022_23.csv")[['play_id','game_id','hash','make_x','make_y']]

pbp_locs = pbp.merge(kicks, on=['play_id', 'game_id'])[['play_id', 'game_id', 'Number', 'Kicker', 'Yards', 'Outcome', 
           'hash','make_x','make_y', 'posteam', 'defteam', 'game_date', 
           'qtr', 'quarter_seconds_remaining', 'yardline_100', 'desc', 
           'stadium', 'time_of_day', 'Team', 'Latitude', 'Longitude',
           'Indoor', 'Turf', 'elevation', 'temperature', 'precipitation',
           'windspeed', 'windgusts']]

display(pbp_locs)

'''
### Old version

kicks = pd.read_csv("STUD Field Goal Tracking - kicks_2022_23.csv")

def create_team_col(offense_team, defense_team):
    a, b = sorted([str(offense_team), str(defense_team)])
    return a + b

kicks['team col'] = kicks.apply(lambda row: create_team_col(row['posteam'], row['defteam']), axis=1)
sked['team col'] = sked.apply(lambda row: create_team_col(row['Home.Abbr'], row['Away.Abbr']), axis=1)

kicks.rename(columns={'GameDate': 'Date'}, inplace=True)

kicks['game_date'] = pd.to_datetime(kicks['game_date'])
kicks.rename(columns={'game_date': 'Date'}, inplace=True)
sked['Date'] = pd.to_datetime(sked['Date'])

final = pd.merge(sked, kicks, on=['Date', 'team col'])

final.to_csv('kicks_with_weather.csv', index=False)
'''

,play_id,game_id,Number,Kicker,Yards,Outcome,hash,make_x,make_y,posteam,...,Team,Latitude,Longitude,Indoor,Turf,elevation,temperature,precipitation,windspeed,windgusts
0,790,2022_01_BAL_NYJ,9,J. Tucker,24,Good,9.25,-6.00,31.0,BAL,...,New York Giants,40.812194,-74.076983,No,Turf,2.0,72.4,0.0,5.2,10.1
1,1216,2022_01_BAL_NYJ,6,G. Zuerlein,45,No Good,9.25,-11.30,28.0,NYJ,...,New York Giants,40.812194,-74.076983,No,Turf,2.0,72.4,0.0,5.2,10.1
2,1919,2022_01_BAL_NYJ,6,G. Zuerlein,45,Good,9.25,-6.90,27.3,NYJ,...,New York Giants,40.812194,-74.076983,No,Turf,2.0,71.7,0.0,5.4,11.2
3,1153,2022_01_BUF_LA,2,T. Bass,41,Good,-9.25,8.00,24.5,BUF,...,Los Angeles Chargers,33.953587,-118.339630,Yes,Turf,40.0,70.0,0.0,0.0,2.5
4,1888,2022_01_BUF_LA,8,M. Gay,57,Good,-9.25,2.80,9.1,LA,...,Los Angeles Chargers,33.953587,-118.339630,Yes,Turf,40.0,70.0,0.0,0.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,3363,2022_21_SF_PHI,4,J. Elliott,31,Good,9.25,2.20,27.8,PHI,...,Philadelphia Eagles,39.900775,-75.167453,No,Hybrid,4.0,44.6,0.0,15.1,27.0
1101,843,2022_22_KC_PHI,7,H. Butker,42,No Good,9.25,-9.25,28.6,KC,...,Arizona Cardinals,33.527700,-112.262608,Retractable,Grass,328.0,70.0,0.0,0.0,2.5
1102,2124,2022_22_KC_PHI,4,J. Elliott,35,Good,-9.25,4.70,33.2,PHI,...,Arizona Cardinals,33.527700,-112.262608,Retractable,Grass,328.0,70.0,0.0,0.0,2.5
1103,2975,2022_22_KC_PHI,4,J. Elliott,33,Good,8.00,-1.30,35.5,PHI,...,Arizona Cardinals,33.527700,-112.262608,Retractable,Grass,328.0,70.0,0.0,0.0,2.5


'\n### Old version\n\nkicks = pd.read_csv("STUD Field Goal Tracking - kicks_2022_23.csv")\n\ndef create_team_col(offense_team, defense_team):\n    a, b = sorted([str(offense_team), str(defense_team)])\n    return a + b\n\nkicks[\'team col\'] = kicks.apply(lambda row: create_team_col(row[\'posteam\'], row[\'defteam\']), axis=1)\nsked[\'team col\'] = sked.apply(lambda row: create_team_col(row[\'Home.Abbr\'], row[\'Away.Abbr\']), axis=1)\n\nkicks.rename(columns={\'GameDate\': \'Date\'}, inplace=True)\n\nkicks[\'game_date\'] = pd.to_datetime(kicks[\'game_date\'])\nkicks.rename(columns={\'game_date\': \'Date\'}, inplace=True)\nsked[\'Date\'] = pd.to_datetime(sked[\'Date\'])\n\nfinal = pd.merge(sked, kicks, on=[\'Date\', \'team col\'])\n\nfinal.to_csv(\'kicks_with_weather.csv\', index=False)\n'

XP% + Age

In [241]:
exp = pd.read_csv("NFL Kicking project - Extra Points.csv")
exp.head()

def format_name(name):
    parts = name.split()  # Split the name into parts
    if len(parts) > 1:  # Check if the name has at least two parts
        first_initial = parts[0][0]  # First letter of the first name
        last_name = parts[-1]  # Last name
        # Remove any non-alphabetic characters from the last name
        last_name_cleaned = ''.join(filter(str.isalpha, last_name)) #.upper()
        return f"{first_initial}. {last_name_cleaned}"
    else:
        return name #.upper()  # Just in case there's a single-part name

# Apply the formatting function to the 'Player' column
exp['Player'] = exp['Player'].apply(format_name)

exp.rename(columns={'Player': 'Kicker'}, inplace=True)

exp['XP%'] = pd.to_numeric(exp['XP%'].str[:-1])

pbp_locs_xp = pbp_locs.merge(exp, on = ['Kicker'])
pbp_locs_xp

,play_id,game_id,Number,Kicker,Yards,Outcome,hash,make_x,make_y,posteam,...,Longitude,Indoor,Turf,elevation,temperature,precipitation,windspeed,windgusts,Age,XP%


Salaries

In [261]:
salaries = pd.read_csv('NFL Kicking project - Salaries.csv')

salaries.rename(columns = {'Player' : 'Kicker',
                           'Salary' : 'cap_hit'},
                           inplace = True)

# Extract initials and last name using regex
salaries['Kicker'] = salaries['Kicker'].str.replace(r'([A-Z\'-])[a-zA-Z\'-]+\s+([A-Z\'-][a-zA-Z\'-]+)\s+[A-Z]+$', r'\1. \2', regex=True)

# Remove newline characters, convert 'Kicker' to uppercase, and 'cap_hit' to integers
salaries['Kicker'] = salaries['Kicker'].str.replace('\n', '').str.strip().apply(format_name)
salaries['cap_hit'] = salaries['cap_hit'].replace('[\$,]', '', regex=True).astype(int)

pbp_locs_xp_sal = pd.merge(pbp_locs_xp, salaries, on = 'Kicker', how = 'left')

pbp_locs_xp_sal.to_csv('final_kick_data_2022.csv', index=False)